Import the libraries so that they can be used within the notebook

* **requests** is used to make HTTP calls
* **json** is used to encode and decode strings into JSON
* **string** is used to perform text manipulation and checking
* **getpass** is used to do non-echoing password input

In [0]:
import requests
import json
import string
import getpass

The **base_url** holds the URL to the SEEK instance that will be used in the notebook

**headers** holds the HTTP headers that will be sent with every HTTP call

* **Content-type: application/vnd.api+json** - indicates that any data sent will be in JSON API format
* **Accept: application/vnd.api+json** - indicates that the notebook expects any data returned to be in JSON API format
* **Accept-Charset: ISO-8859-1** - indicates that the notebook expects any text returned to be in ISO-8859-1 character set

In [0]:
base_url = 'https://testing.sysmo-db.org'

headers = {"Content-type": "application/vnd.api+json",
           "Accept": "application/vnd.api+json",
           "Accept-Charset": "ISO-8859-1"}

Create a **requests** HTTP **Session**. A **Session** has re-usable settings such as **headers**

The **authorization** is username and password. The user is prompted for this information.

In [0]:
session = requests.Session()
session.headers.update(headers)
session.auth = (input('Username: '), getpass.getpass('Password: '))

The **Investigation**, **Study** and **Assay** will be created within **Project** 33

In [0]:
containing_project_id = 33


Initialize an **investigation** as a hierarchical structure

The title of the **investigation** is input by the user

The **investigation** is linked to the containing **project**

In [0]:
investigation = {}
investigation['data'] = {}
investigation['data']['type'] = 'investigations'
investigation['data']['attributes'] = {}
investigation['data']['attributes']['title'] = input('Please enter the name for the investigation: ')
investigation['data']['relationships'] = {}
investigation['data']['relationships']['projects'] = {}
investigation['data']['relationships']['projects']['data'] = [{'id' : containing_project_id, 'type' : 'projects'}]


**POST** the **investigation** to the SEEK instance

Check the status of the response

In [0]:
r = session.post(base_url + '/investigations', json=investigation)
r.raise_for_status()

Extract the created **investigation** from JSON into **populated_investigation**

In [0]:
populated_investigation = r.json()

Extract the id and URL to the newly created **investigation**

In [0]:
investigation_id = populated_investigation['data']['id']
investigation_url = populated_investigation['data']['links']['self']

Initialize a **study** as a hierarchical structure

The title of the **study** is input by the user

The **study** is linked to the containing **investigation**

In [0]:
study = {}
study['data'] = {}
study['data']['type'] = 'studies'
study['data']['attributes'] = {}
study['data']['attributes']['title'] = input('Please enter the name for the study: ')
study['data']['relationships'] = {}
study['data']['relationships']['investigation'] = {}
study['data']['relationships']['investigation']['data'] = {'id' : investigation_id, 'type' : 'investigations'}

**POST** the **study** to the SEEK instance

Check the status of the response

In [0]:
r = session.post(base_url + '/studies', json=study)
r.raise_for_status()

Extract the created **study** from JSON into ***populated_study***

In [0]:
populated_study = r.json()

Extract the id and URL to the newly created **study**

In [0]:
study_id = populated_study['data']['id']
study_url = populated_study['data']['links']['self']

Initialize an **assay** as a hierarchical structure

The title of the **assay** is input by the user

The **assay** is linked to the containing **study**

In [0]:
assay = {}
assay['data'] = {}
assay['data']['type'] = 'assays'
assay['data']['attributes'] = {}
assay['data']['attributes']['title'] = input('Please enter the name for the assay: ')
assay['data']['attributes']['assay_class'] = {'key' : 'EXP'}
assay['data']['attributes']['assay_type'] = {'uri' : 'http://jermontology.org/ontology/JERMOntology#Metabolomics'}
assay['data']['attributes']['technology_type'] = {'uri' : 'http://jermontology.org/ontology/JERMOntology#Electrophoresis'}
assay['data']['relationships'] = {}
assay['data']['relationships']['study'] = {}
assay['data']['relationships']['study']['data'] = {'id' : study_id, 'type' : 'studies'}

**POST** the **assay** to the SEEK instance

Check the status of the response

In [0]:
r = session.post(base_url + '/assays', json=assay)
r.raise_for_status()

Extract the created **assay** from JSON into ***populated_assay***

In [0]:
populated_assay = r.json()

Extract the id and URL to the newly created **assay**

In [0]:
assay_id = populated_assay['data']['id']
assay_url = populated_assay['data']['links']['self']

Initialize a **data_file** as a hierarchical structure

The title of the **data_file** is input by the user

The **data_file** is linked to the containing **project** and to the newly created **assay**

In [0]:
data_file = {}
data_file['data'] = {}
data_file['data']['type'] = 'data_files'
data_file['data']['attributes'] = {}
data_file['data']['attributes']['title'] = input('Please enter the name for the data_file: ')

remote_blob = {'url' : input('Please enter the URL for the remote data: ')}
data_file['data']['attributes']['content_blobs'] = [remote_blob]
data_file['data']['relationships'] = {}
data_file['data']['relationships']['projects'] = {}
data_file['data']['relationships']['projects']['data'] = [{'id' : containing_project_id, 'type' : 'projects'}]
data_file['data']['relationships']['assays'] = {}
data_file['data']['relationships']['assays']['data'] = [{'id' : assay_id, 'type' : 'assays'}]

**POST** the **data_file** to the SEEK instance

Check the status of the response

In [0]:
r = session.post(base_url + '/data_files', json=data_file)
r.raise_for_status()

Extract the created **data_file** from JSON into **populated_data_file**

In [0]:
populated_data_file = r.json()

Extract the id and URL to the newly created **data_file**

In [0]:
data_file_id = populated_data_file['data']['id']
data_file_url = populated_data_file['data']['links']['self']